# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
import langchain

In [1]:
import os
import dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

dotenv.load_dotenv()

True

In [2]:
loader = TextLoader('data.txt', encoding='utf-8')
doc = loader.load()
doc

[Document(metadata={'source': 'data.txt'}, page_content="Who you are?\nYou are a chatbot designed for ACME Technologies Ltd to answer HR-related questions. Acme Technologies Ltd, Org, started as a modest real estate developer by a group of retired military officers and engineers. Later it entered into the construction of industrial and commercial complexes with a different name called Acme Constructions. By following basic business ethics, like, timely delivery, quality of work, and stunning architecture, Acme Technologies Ltd came into the limelight. \\n\\nTo provide its own needs and the needs of the clients it started a business in the service sector with a separate company called Acme Services. \\n\\nOf late, it also extended its business in the field of information technology. Its venture in IT is named Acme AI. It’s mainly doing annotation now but in the future, it plans to enter the other fields of IT to be a part of smart Bangladesh.\nWhat is the meaning of HR?,\nHR stands for 

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(doc)
texts = texts[:50]

In [4]:
len(texts)

22

In [5]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb

In [6]:
client = chromadb.PersistentClient(path='vectordb')
collection = client.get_or_create_collection(name='hr_manual')


In [7]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\shahabuddin akhon hr\AppData\Local\Temp\ipykernel_15348\2213097150.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\shahabuddin akhon hr\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\shahabuddin akhon hr\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
chroma_db = Chroma.from_documents(
documents=texts,
embedding=embedding_model,
persist_directory='vectordb',
collection_name='hr_manual'
)

In [9]:
vectordb = Chroma(persist_directory='vectordb', embedding_function=embedding_model)

C:\Users\shahabuddin akhon hr\AppData\Local\Temp\ipykernel_15348\3944112970.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory='vectordb', embedding_function=embedding_model)


Make a retriever

In [10]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
retriever.get_relevant_documents("What is HR")

C:\Users\shahabuddin akhon hr\AppData\Local\Temp\ipykernel_15348\1917761674.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is HR")


[]

In [11]:
retriever.search_kwargs

{'k': 2}

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model="llama-3.1-8b-instant",
)

In [13]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [14]:
def process_llm_response(llm_response):
    print(llm_response['result'])

In [18]:
# full example
query = "Tell me about acme technologies"
llm_response = qa_chain(query)
process_llm_response(llm_response)

I don't have any information about a company called "Acme Technologies." However, I do know that "Acme" is a common name for fictional companies in cartoons and comics, often used as a generic placeholder for a generic or humorous company. 

One example is the Acme Corporation from the Looney Tunes cartoons, which is often depicted as a supplier of outlandish and often malfunctioning products.
